# 2D Advection-Diffusion equation

in this notebook we provide a simple example of the DeepMoD algorithm and apply it on the 2D advection-diffusion equation. 

In [1]:
# General imports
import numpy as np
import torch
# DeepMoD functions
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library2D_third
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold,PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from scipy.io import loadmat

# Settings for reproducibility
np.random.seed(1)
torch.manual_seed(1)

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## Prepare the data

Next, we prepare the dataset.

In [2]:
data = loadmat('Diffusion_2D_space41.mat')
data = np.real(data['Expression1']).reshape((41,41,41,4))[:,:,:,3]
x_dim, y_dim, t_dim = data.shape

In [3]:
time_range = [0.01,0.02,0.05,0.1,0.15,0.2,0.3,0.5,0.75,1,1.5,2,3,4]

In [4]:
for i in time_range:
    
    # Downsample data and prepare data without noise:
    down_data= np.take(np.take(np.take(data,np.arange(0,x_dim,4),axis=0),np.arange(0,y_dim,4),axis=1),np.arange(0,t_dim,3),axis=2)
    print("Dowmsampled shape:",down_data.shape, "Total number of data points:", np.product(down_data.shape))
    index = len(np.arange(0,t_dim,i))    
    width, width_2, steps = down_data.shape
    x_arr, y_arr, t_arr = np.linspace(0,1,width), np.linspace(0,1,width_2), np.linspace(0,1,steps)
    x_grid, y_grid, t_grid = np.meshgrid(x_arr, y_arr, t_arr, indexing='ij')
    X, y = np.transpose((t_grid.flatten(), x_grid.flatten(), y_grid.flatten())), np.float32(down_data.reshape((down_data.size, 1)))
    
    
    # Add noise 
    noise_level = i
    y_noisy = y + noise_level * np.std(y) * np.random.randn(y.size, 1)

    # Randomize data 

    idx = np.random.permutation(y.shape[0])
    X_train = torch.tensor(X[idx, :], dtype=torch.float32, requires_grad=True).to(device)
    y_train = torch.tensor(y_noisy[idx, :], dtype=torch.float32).to(device)

    # Configure DeepMoD

    network = NN(3, [40, 40, 40, 40], 1)
    library = Library2D_third(poly_order=0) 
    estimator = Threshold(0.05) 
    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
    constraint = LeastSquares() 
    model = DeepMoD(network, library, estimator, constraint).to(device)
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=2e-3) 
    logdir='final_runs/Noise_runs_11_11_14/'+str(i)+'/'
    train(model, X_train, y_train, optimizer,sparsity_scheduler, log_dir=logdir, split=0.8, max_iterations=50000, delta=1e-6, patience=200) 


Dowmsampled shape: (11, 11, 14) Total number of data points: 1694
 49975  MSE: 4.52e-05  Reg: 6.02e-06  L1: 1.01e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (11, 11, 14) Total number of data points: 1694
 49975  MSE: 2.57e-05  Reg: 2.99e-06  L1: 1.04e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (11, 11, 14) Total number of data points: 1694
 49975  MSE: 2.85e-05  Reg: 3.00e-06  L1: 1.25e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (11, 11, 14) Total number of data points: 1694
 49975  MSE: 3.23e-05  Reg: 4.60e-06  L1: 1.01e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (11, 11, 14) Total number of data points: 1694
 49975  MSE: 5.06e-05  Reg: 2.60e-06  L1: 1.20e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (11, 11, 14) Total number of data points: 1694
 49025  MSE: 1.60e-04  Reg: 1.06e-05  L1: 1.00e+00 Algorithm converged. Writing model to disk.
Dowmsampled shape: (11, 11, 14) To